In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load in csv from part 1
city_data= pd.read_csv("../WeatherPy/city_data.csv")
city_data.dropna(inplace = True)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Olafsvik,64.8945,-23.7142,49.73,69,64,9.71,IS,1595451600
1,1,Beamsville,43.1668,-79.4829,79.81,74,95,8.30,CA,1595451600
2,2,Bredasdorp,-34.5322,20.0403,51.31,92,0,3.06,ZA,1595451600
3,3,Hong Kong,22.2855,114.1577,84.29,76,47,6.96,HK,1595451600
4,4,Butaritari,3.0707,172.7902,83.41,75,52,15.19,KI,1595451600
...,...,...,...,...,...,...,...,...,...,...
562,562,Anajatuba,-3.2644,-44.6197,85.03,50,37,2.06,BR,1595451600
563,563,Poshekhon'ye,58.4993,39.1353,55.54,90,40,6.60,RU,1595451600
564,564,Itagibá,-14.2836,-39.8428,74.53,56,2,4.59,BR,1595451600
565,565,Babaeski,41.4325,27.0931,72.05,66,0,8.46,TR,1595451600


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations= city_data[["Lat", "Lng"]]
humidity= city_data["Humidity"].astype(float)

In [5]:
# Add Heatmap layer to map.
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create new DataFrame to narrow down cities by ideal weather conditions 
# Max temperature lower than 80 but higher than 70. 
ideal_conditions= city_data.loc[city_data["Max Temp"]<80,:]
ideal_conditions = ideal_conditions.loc[ideal_conditions["Max Temp"]>70,:]

# Wind speed less than 10 mph.
ideal_conditions = ideal_conditions.loc[ideal_conditions["Wind Speed"]<10,:]

#Zero cloudiness.
ideal_conditions = ideal_conditions.loc[ideal_conditions["Cloudiness"]==0,:]
ideal_conditions.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,Farah,32.5000,62.5000,75.90,19,0,1.77,AF,1595451600
123,123,Port Said,31.2565,32.2841,79.65,80,0,8.88,EG,1595451600
162,162,Arraial do Cabo,-22.9661,-42.0278,74.66,78,0,8.52,BR,1595451600
304,304,Bani Walid,31.7566,13.9942,74.44,63,0,7.54,LY,1595451600
328,328,Lata,40.1629,-8.3327,74.28,55,0,2.13,PT,1595451600


In [7]:

#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

In [8]:
#Hotel Map
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
hotel_df['City'] = ideal_conditions['City']
hotel_df['Country'] = ideal_conditions['Country']
hotel_df['Lat'] = ideal_conditions['Lat']
hotel_df['Lng'] = ideal_conditions['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
25,Farah,AF,32.5000,62.5000
123,Port Said,EG,31.2565,32.2841
162,Arraial do Cabo,BR,-22.9661,-42.0278
304,Bani Walid,LY,31.7566,13.9942
328,Lata,PT,40.1629,-8.3327


In [9]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 25: Farah.
Missing field/result... skipping.
------------
Retrieving Results for Index 123: Port Said.
Closest hotel to Port Said at 31.2565 , 32.2841 is JOCKEY.
------------
Retrieving Results for Index 162: Arraial do Cabo.
Closest hotel to Arraial do Cabo at -22.9661 , -42.0278 is HI Marina dos Anjos.
------------
Retrieving Results for Index 304: Bani Walid.
Closest hotel to Bani Walid at 31.7566 , 13.9942 is University of Bani Walid General Administration.
------------
Retrieving Results for Index 328: Lata.
Closest hotel to Lata at 40.1629 , -8.3327 is Atocentro Fabrico De Acessorios Para A Agricultura, Lda..
------------
Retrieving Results for Index 484: Yabrūd.
Closest hotel to Yabrūd at 33.9674 , 36.6574 is المركز الهندسي التقني - م.ضياء الدين الأجرد.
------------
Retrieving Results for Index 527: Vinh.
Closest hotel to Vinh at 18.6667 , 105.6667 is Ngân hàng Vietinbank.
------------
Retrieving Results for Index 550: Saquarema.
Closest hotel to Saq

In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
25,Farah,AF,32.5000,62.5000,NaN
123,Port Said,EG,31.2565,32.2841,JOCKEY
162,Arraial do Cabo,BR,-22.9661,-42.0278,HI Marina dos Anjos
304,Bani Walid,LY,31.7566,13.9942,University of Bani Walid General Administration
328,Lata,PT,40.1629,-8.3327,Atocentro Fabrico De Acessorios Para A Agricul...
484,Yabrūd,SY,33.9674,36.6574,المركز الهندسي التقني - م.ضياء الدين الأجرد
527,Vinh,VN,18.6667,105.6667,Ngân hàng Vietinbank
550,Saquarema,BR,-22.9200,-42.5103,Hospedagem Terral
565,Babaeski,TR,41.4325,27.0931,Beko


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [12]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))